In [1]:
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_datetime64_ns_dtype
import inflect

In [2]:
def populate_org_dict(target_df):
    result = {}

    for i in range(len(target_df)):
        org_id = target_df.loc[i, 'organization_id']
        org_id = int(org_id) if not pd.isnull(org_id) else 0

        if org_id in result:
            result[org_id] += ', ' + str(target_df.loc[i, '_id'])
        else:
            result[org_id] = str(target_df.loc[i, '_id'])
    return result

In [3]:
def generate_org_full_table(ticket_dict, user_dict):
    try:
        df_ticket_dict = pd.DataFrame(ticket_dict.items(), columns=['_id', 'tickets'])
        df_user_dict = pd.DataFrame(user_dict.items(), columns=['_id', 'users'])
        df_orgs_tmp = df_orgs.copy(deep=True)

        return pd.merge(pd.merge(df_orgs_tmp, df_ticket_dict, how='left', on='_id'), df_user_dict, how='left', on='_id')
    except BaseException as error:
        print('An exception occurred: {}'.format(error))
        return False

In [4]:
def generate_user_tkt_full_table(table_name, table_df, org_dict):
    new_column_name = 'users' if table_name == 'tickets' else 'tickets'
    table_df[new_column_name] = [list() for x in range(len(table_df.index))]                     
    
    for i in range(len(table_df)):
        organization_id = table_df.loc[i, 'organization_id']
        if organization_id in org_dict:
            table_df.at[i, new_column_name] = org_dict[organization_id].split(', ')
    
    return table_df

In [5]:
df_tickets = pd.read_json('tickets.json')
df_users = pd.read_json('users.json')
df_orgs = pd.read_json('organizations.json')

org_to_tkt_dict = populate_org_dict(df_tickets)
df_users_full = generate_user_tkt_full_table('users', df_users.copy(deep=True), org_to_tkt_dict)

org_to_user_dict = populate_org_dict(df_users)
df_tickets_full = generate_user_tkt_full_table('tickets', df_tickets.copy(deep=True), org_to_user_dict)

df_orgs_full = generate_org_full_table(org_to_tkt_dict, org_to_user_dict)

query_table = None
query_column = None
query_value = None

In [6]:
def get_column_type(target_df, column):
    if column in target_df.columns:
        if is_string_dtype(target_df[column]):
            return 'string'
        elif is_numeric_dtype(target_df[column]):
            return 'int'
        elif is_datetime64_ns_dtype(target_df[column]):
            return 'datetime'
        else:
            return 'unknown'
    else:
        return 'unknown'

In [7]:
def print_formatted_result(result):
    n_ordinal = inflect.engine()
    
    if result.empty:
        print('Search {0} for {1} with a value of {2}\nNo results found'.format(query_table, query_column, query_value[1]))
    else:
        columns = list(result.columns)
        for i in range(len(result)):
            for j in range(len(columns)):
                print('{0}: {1}\n'.format(columns[j], result.iloc[i, j]))
            print('====== End of the %s result ======' % n_ordinal.ordinal(i+1))

In [8]:
def equal_query(target_df, column, column_type, value):
    global query_result
    
    if column_type == 'int':
        value = int(value)

    query_result = target_df[target_df[column] == value]

In [9]:
def like_query(target_df, column, value):
    global query_result
    
    query_result = target_df[target_df[column].str.contains(value, na=False)]

In [10]:
def parse_query(input_2, input_3, input_4):
    CONST_TABLES_DICT = {1: ['users', 'users'], 2: ['tickets', 'tickets'], 3: ['orgs', 'organizations']}
    
    target_df = globals()['df_' + CONST_TABLES_DICT[input_2][0] + '_full']
    
    global query_table
    query_table = CONST_TABLES_DICT[input_2][1]
    
    global query_column
    query_column =  input_3
    
    global query_value
    query_value = input_4.split('@@')
    
    column_type = get_column_type(target_df, query_column)
    
    if query_value[0] == 'equal':
        equal_query(target_df, query_column, column_type, query_value[1])
    elif query_value[0] == 'like':
        like_query(target_df, query_column, query_value[1])
    ## To-DO: Add more type of query functions, and also will support multiple columns query

In [11]:
query_result = None
parse_query(2, '_id', 'equal@@916aab4a-0577-40cf-8f56-a45912a6ac23')
print_formatted_result(query_result)

_id: 916aab4a-0577-40cf-8f56-a45912a6ac23

url: http://initech.zendesk.com/api/v2/tickets/916aab4a-0577-40cf-8f56-a45912a6ac23.json

external_id: 9fbaddb4-d72f-4270-baf9-dba1de59bb4b

created_at: 2016-05-14T11:36:47 -10:00

type: task

subject: A Nuisance in Togo

description: Magna dolor do enim velit incididunt reprehenderit aute consequat irure fugiat ullamco. Nulla tempor occaecat consectetur culpa consectetur proident culpa.

priority: low

status: pending

submitter_id: 19

assignee_id: 39.0

organization_id: 110.0

tags: ['Mississippi', 'Marshall Islands', 'South Dakota', 'Montana']

has_incidents: False

due_at: 2016-08-15 01:19:11-10:00

via: chat

users: ['12', '21', '24', '37', '56']

====== End of the 1st result ======


In [12]:
parse_query(3, '_id', 'equal@@110')
print_formatted_result(query_result)

_id: 110

url: http://initech.zendesk.com/api/v2/organizations/110.json

external_id: 197f93c0-1729-4c82-9bb0-143e978f06ce

name: Kindaloo

domain_names: ['translink.com', 'netropic.com', 'earthplex.com', 'zilencio.com']

created_at: 2016-03-15T03:08:47 -11:00

details: Non profit

shared_tickets: True

tags: ['Chen', 'Melton', 'Stafford', 'Landry']

tickets: 916aab4a-0577-40cf-8f56-a45912a6ac23, c496e355-4400-4baa-b8ca-bb2edd270c43, 6ed590ac-e385-46e2-a27a-50628a658168, ccf4c82c-f572-4fd2-82a6-11d6055929b8, 140e0cd4-c31b-4e90-833d-c42a12d4b713

users: 12, 21, 24, 37, 56

====== End of the 1st result ======


In [13]:
parse_query(1, 'alias', 'like@@Mr')
print_formatted_result(query_result)

_id: 4

url: http://initech.zendesk.com/api/v2/users/4.json

external_id: 37c9aef5-cf01-4b07-af24-c6c49ac1d1c7

name: Rose Newton

alias: Mr Cardenas

created_at: 2016-02-09T07:52:10 -11:00

active: True

verified: 1.0

shared: True

locale: de-CH

timezone: Netherlands

last_login_at: 2012-09-25T01:32:46 -10:00

email: cardenasnewton@flotonic.com

phone: 8685-482-450

signature: Don't Worry Be Happy!

organization_id: 122.0

tags: ['Gallina', 'Glenshaw', 'Rowe', 'Babb']

suspended: True

role: end-user

tickets: ['c73a0be5-e967-4948-b0a4-eff98d1a43ad', 'e33110bb-fd7b-4983-987a-4172a9e24919', '8629d5fa-89c4-4e9b-9d9f-221b68b079f4', 'a28d5e97-ab21-44ef-b4c4-95105a75e184', '7251d3d2-a735-487d-9481-243c3048f171', 'cf0d4a27-0dcb-49a9-a4fd-beec25742799', 'c45893d9-17c2-43b0-8800-a5f8201aff93', '1153a9d0-80b8-45f8-9753-e1c004caea7b', 'f75ef2ed-da4f-417c-b164-3dd2c9c8f87c', '9fe171f6-8790-4d8c-9463-b90052ee7423', '189eed9f-b44c-49f3-a904-2c482193996a', 'fd26f66a-5688-43ad-8890-c3d65f84c6c0', 

In [14]:
parse_query(1, 'name', 'equal@@George')
print_formatted_result(query_result)

Search users for name with a value of George
No results found
